In [1]:
from konlpy.tag import Twitter
from gensim.models import Word2Vec
import csv
from konlpy.tag import Twitter
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import tensorflow as tf
import matplotlib

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

import csv
import logging
from konlpy import jvm
logger = logging.getLogger(__name__)
jvm.init_jvm()



C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages

In [2]:
data_set = pd.read_excel("data.xlsx")
data_set = data_set.iloc[:,1:]
data_set.rename({0:'news'},axis=1, inplace= True)
data_label = {1:'culture',2:'global',3:'politic',4:'society',5:'economy'}




In [3]:
for i in range(len(data_set)) :
    data_set['news'][i] = re.sub('[^a-zA-Z0-9 ㄱ-ㅣ가-힣]', '', data_set['news'][i])
    


C:\Users\ICT01_21\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
data_set = data_set.sample(frac=1).reset_index(drop=True)

In [5]:
data_set = data_set.iloc[:40000,:]

In [6]:
data_set.head()

,news,label
0,1810만명이 민주당 경선 보자 트럼프 DMZ 흥행쇼 던졌다,2
1,단독 건보공단 N분의1 성과급 안고치면 내년엔 안준다,5
2,올 최강 태풍 하기비스 일본 강타할 듯동해안 간접영향,1
3,스브스뉴스 고래사냥 다시 시작한 그들을 막아선 해적,2
4,한국당 뺀 여야 518 논란 김진태이종명김순례 징계 추진,3


In [7]:
data_set.isna().sum()

news     0
label    0
dtype: int64

In [8]:
def get_noun(text):
    tokenizer = Twitter()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_noun2(text):
    tokenizer = Hannanum()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_noun3(text):
    tokenizer = Okt()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_morphs3(text):
    tokenizer = Okt()
    morphs = tokenizer.morphs(text)
    return [n for n in morphs]

def get_noun4(text):
    tokenizer = Komoran()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]


In [9]:
train_set, test_set = train_test_split(data_set, test_size = 0.3, random_state = 42)

In [10]:
train_set=train_set.reset_index(drop=True)
test_set=test_set.reset_index(drop=True)

In [56]:
#fit_transform = CV.fit_transform(train_set["news"])

In [11]:
#label_set = data_set['label']
label_set = train_set['label']
len(label_set)

28000

In [12]:
train_set

,news,label
0,유사군복 판매자 처벌 합헌 결정 후폭풍,3
1,북한 목선 책임 8군단장 해임합참의장 등 엄중 경고,3
2,교직 관두고 일 도우라 부친 부름에글로벌 기업으로 이끈 구자,5
3,월드리포트 시진핑은 정말 평양 답방을 주저하는 걸까,2
4,베트남 아내 짓밟은 한국인 남편참다 못해 몰래 찍었다,4
...,...,...
27995,비하인드 뉴스 이낙연 총리가 직접 밝힌 큰절의 이유,3
27996,7인조 아이돌 머스트비 교통사고운전하던 매니저 사망,4
27997,화성연쇄살인 용의자 몽타주와 다르다처제 살인 때 못 알아봐,4
27998,정경심 교수 구속법원 범죄 혐의 상당 부분 소명,4


In [13]:
test_set

,news,label
0,SUV차량 하천 다리 밑 추락필리핀행 여객기 회항,3
1,BMW서 헬기 수준 굉음 고통 호소업체 결함 아냐,4
2,작년에 산 롱패딩이 무색하게올 겨울 왜 이상고온일까,1
3,독도서 첫 블랙 워터 다이빙한밤 수중 생물 조사,1
4,흔들리는 무기 빅딜 훈련기 수출 백지화 우려도 박수찬의,3
...,...,...
11995,군산에 묻어달라던 서양인이 125년 전 남긴 사진,1
11996,전광훈 황교안 정치가 아니다아무리 가르쳐도 안돼,1
11997,한 명이라도 더 살려야끝까지 현장 지킨 응급의료 버팀목,4
11998,단독 의사 접대 자리에서 성폭력임직원은 쉬쉬,4


In [111]:
from konlpy.tag import Twitter
from konlpy.tag import Kkma
tf.reset_default_graph()
twitter = Twitter()
token = []
embeddingmodel = []
for i in range(len(data_set)):
    label = data_set['label']
    sentence = twitter.pos(data_set['news'][i], norm=True, stem=True)
    temp = []
    temp_embedding = []
    all_temp = []
    for k in range(len(sentence)):
        temp_embedding.append(sentence[k][0])
        temp.append(sentence[k][0] + '/' + sentence[k][1])
    all_temp.append(temp)
    embeddingmodel.append(temp_embedding)
    
    all_temp.append(label[i])
    token.append(all_temp)
    
embeddingmodel = []
for i in range(len(label_set)):
    temp_embeddingmodel = []
    for k in range(len(token[i][0])):
        temp_embeddingmodel.append(token[i][0][k])
    embeddingmodel.append(temp_embeddingmodel)
embedding  = Word2Vec(embeddingmodel, size = 300, window =5, min_count = 10,
                      iter = 5, sg=1)

C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [112]:
def One_hot(data):
       
    index_dict = {value:index for index,value in enumerate(set(data))}
    result = []

    for value in data:

        one_hot = np.zeros(len(index_dict))
        index = index_dict[value]
        one_hot[index] = 1
        result.append(one_hot)

    return result


def Convert2Vec(model_name, doc): 
    #train_X_ = W2V.Convert2Vec("Word2Vec_csv_article.embedding",train_X)
    word_vec = []
    model = model_name
    for sent in doc:
        sub = []
        for word in sent:
            if word in model.wv.vocab:
                sub.append(model.wv[word]) 
            else:
                sub.append(np.random.uniform(-0.25,0.25,300)) 
        word_vec.append(sub)

    return word_vec





In [113]:
class Bi_LSTM():
    
    def __init__(self, lstm_units, num_class, keep_prob):
        
        self.lstm_units = lstm_units
        
        with tf.variable_scope('forward', reuse = tf.AUTO_REUSE):
            
            self.lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
            
        with tf.variable_scope('backward', reuse = tf.AUTO_REUSE):
            
            self.lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
        
        with tf.variable_scope('Weights', reuse = tf.AUTO_REUSE):
           
            self.W = tf.get_variable(name="W", shape=[2 * lstm_units, num_class],
                                dtype=tf.float32, initializer = tf.contrib.layers.xavier_initializer())
            self.b = tf.get_variable(name="b", shape=[num_class], dtype=tf.float32,
                                initializer=tf.zeros_initializer())
            
            
    def logits(self, X, W, b, seq_len):
        
        (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(self.lstm_fw_cell, self.lstm_bw_cell,dtype=tf.float32,
                                                                            inputs = X, sequence_length = seq_len)
        ## concat fw, bw final states
        outputs = tf.concat([states[0][1], states[1][1]], axis=1)
        pred = tf.matmul(outputs, W) + b        
        return pred
        
    def model_build(self, logits, labels, learning_rate = 0.001):
        
        with tf.variable_scope("loss"):
            
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits , labels = labels)) # Softmax loss
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) 
            
        return loss, optimizer
    
    def graph_build(self, avg_loss, avg_acc):
        
        tf.summary.scalar('Loss', avg_loss)
        tf.summary.scalar('Accuracy', avg_acc)
        merged = tf.summary.merge_all()
        return merged

In [114]:
def Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size):
        
    zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))
    for i in range(Batch_size):
        zero_pad[i, :np.shape(train_batch_X[i])[0], :np.shape(train_batch_X[i])[1] ] = train_batch_X[i]
    return zero_pad

In [115]:
tokens = np.array(token)
data_x = tokens[:,0]
data_y = tokens[:,1]


data_y = One_hot(data_y)
data_x = Convert2Vec(embedding, data_x)

train_x, test_x, train_y, test_y = train_test_split(data_x,data_y,
                                                    test_size = 0.3,
                                                    random_state = 42)
print(len(train_x))
print(len(train_y))
print(len(test_x))
#train_x = train_x.reset_index(drop=True)
#train_y = train_y.reset_index(drop=True)
#test_x = test_x.reset_index(drop=True)
#test_y = test_y.reset_index(drop=True)

28000
28000
12000


In [116]:
len(data_x[0][0])

300

In [120]:
Batch_size = 32
Total_size = len(train_x)
Vector_size = 300
seq_length = [len(x) for x in train_x]
Maxseq_length = max(seq_length)
learning_rate = 0.001
lstm_units = 128
num_class = 5
training_epochs = 20
keep_prob = 0.75

X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size])
Y = tf.placeholder(tf.float32, shape = [None, num_class])
seq_len = tf.placeholder(tf.int32 , shape = [None])

BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)
with tf.variable_scope('loss', reuse = tf.AUTO_REUSE) :
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)

prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

total_batch = int(Total_size / Batch_size)

print("Start training!")

#modelName = "BiLSTM.model"
saver = tf.train.Saver()

Start training!


In [121]:
test_set

,news,label
0,미 항공청보잉 결함 알고도 무시보잉 게이트로 번지나,2
1,이미선 헌법재판관 임명은 국민 개무시 한국당 정부 규탄 집회,3
2,이지영 강사 학생들 상대로 천효재단 포교 논란,1
3,정글의 법칙 취식 사과한 대왕조개 100년 사는 멸종위기종,2
4,단독 파업 312시간 일 않고도임금 보전받는 르노삼성 노조,5
...,...,...
11995,커피 놀이터가 된 잠실커피 잔만 바꿔도 나만의 스타일 완성되죠,1
11996,인천 중학생 추락사 가해학생 4명은 왜 모두 실형을 받았나,4
11997,송중기송혜교 이혼조정신청드라마 남자친구 박보검과 다정한 모,4
11998,16세 정치 입문세계 최연소 총리 타이틀 거머 쥔 쿠르츠,2


In [ ]:
import time
import os
result  = []
with tf.Session() as sess:

    start_time = time.time()
    sess.run(init)
    train_writer = tf.summary.FileWriter('Bidirectional_LSTM', sess.graph)
    i = 0
    for epoch in range(training_epochs):
        #try :
            avg_acc, avg_loss = 0. , 0.
            for step in range(total_batch):

                train_batch_X = train_x[step*Batch_size : step*Batch_size+Batch_size]

                train_batch_Y = train_y[step*Batch_size : step*Batch_size+Batch_size]
                batch_seq_length = seq_length[step*Batch_size : step*Batch_size+Batch_size]
                train_batch_X = Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)

                sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
                
                loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
                avg_loss += loss_ / total_batch

                acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
                avg_acc += acc / total_batch
            print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))
            #print("Test Accuracy : ", sess.run(accuracy, feed_dict={X:test_x, Y:test_y}))
            summary = sess.run(BiLSTM.graph_build(avg_loss, avg_acc))       
            train_writer.add_summary(summary, i)
            i += 1
        #except : 
            #print('Error Epoch : ', epoch)
    duration = time.time() - start_time
    minute = int(duration / 60)
    second = int(duration) % 60
    print("%dminutes %dseconds" % (minute,second))
    save_path = saver.save(sess, os.getcwd())

    train_writer.close()
    print('save_path',save_path)
    
    test_size = len(test_x)
    test_batch = int(test_size / Batch_size)
    seq_length = [len(x) for x in test_x]
    keep_prob = 1.0
    total_acc = 0
    for step in range(test_batch):
        test_batch_X = test_x[step*Batch_size: step*Batch_size + Batch_size]
        test_batch_Y = test_y[step*Batch_size: step*Batch_size + Batch_size]
        batch_seq_length = seq_length[step*Batch_size : step*Batch_size + Batch_size]
        test_batch_X = Zero_padding(test_batch_X, Batch_size, Maxseq_length, Vector_size)
        
        #summary, _ = sess.run([mreged, optimizer], feed_dict = {X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length})
        acc = sess.run(accuracy , feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length})
        total_acc += acc/test_batch
        pred_v = sess.run(prediction , feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length})
    #print('acc : ', acc)
        
        if pred_v is None :
                    pred_v = np.zeros(shape=(32,2))
        
        X_batch = test_x[step*Batch_size : step*Batch_size+Batch_size]
        pred_df_row = pd.DataFrame(list(zip(X_batch, test_batch_Y, pred_v)),columns = ["test_X",'test_Y','pred'])
                
        if step == 0:
            pred_df = pred_df_row
        else:
            pred_df = pd.concat([pred_df, pred_df_row], axis = 0)
            
        if (step > 10 and step & 100 == 0) or (step == test_batch - 1):
            loss_ = sess.run(loss, feed_dict ={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length})
            acc = sess.run(accuracy , feed_dict = {X:test_batch_X, Y:test_batch_Y, seq_len: batch_seq_length})
            
            #time_str = datetime.datetime.new().isoforamt()
            print("loss = {:.6f} accuracy = {:.6f}".format(loss_, acc), "/ step =",step)
            #test_writer.add_summary(summary, step)
            
    print("\nepoch : ", epoch + 1, " - Total Accuracy : {}".format(total_acc))
    result.append(total_acc)

    #pred_df_file = out_dir + "/pred_value_" + str(epoch + 1)+'.csv'
    #pred_df.to_csv(pred_df_file, index=False, header=False)
    #print("epoch : ",epoch+1,"- Saved pred_value : {}\n".format(pred_df_file))
        

print("----------------------------------------")
print("????????")
for i in range(0, len(result)):
    print("epoch : ", i+1, " / accuracy : ", result[i])
    
print("\nBest Result epoch : ", result.index(max(result)) + 1, " / accuracy: ", max(result))

epoch : 01 step : 0875 loss = 0.803297 accuracy= 0.625000
epoch : 02 step : 0875 loss = 0.761300 accuracy= 0.656250
epoch : 03 step : 0875 loss = 0.729546 accuracy= 0.718750
epoch : 04 step : 0875 loss = 0.708430 accuracy= 0.718750
epoch : 05 step : 0875 loss = 0.674488 accuracy= 0.718750
epoch : 06 step : 0875 loss = 0.612798 accuracy= 0.750000
epoch : 07 step : 0875 loss = 0.544965 accuracy= 0.843750
epoch : 08 step : 0875 loss = 0.473899 accuracy= 0.875000
epoch : 09 step : 0875 loss = 0.389920 accuracy= 0.875000
epoch : 10 step : 0875 loss = 0.310277 accuracy= 0.906250


In [ ]:
#Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)
#print(train_batch_X[1].shape)
zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))
print(zero_pad.shape)
print(np.shape(train_batch_X[0]))

#print(zero_pad.shape[0,list(np.shape(train_batch_X[0]))[0],:])
a,b = 0,0
for i in range(Batch_size):
    a= int(np.shape(train_batch_X[i])[0])
    b= int(np.shape(train_batch_X[i])[1])
    zero_pad[i,:a,:b] = train_batch_X[i]
zero_pad[:10]

In [ ]:
np.shape(train_batch_X[1])[1]

In [ ]:
np.asarray(train_batch_X[0]).shape[1]

In [ ]:
int(np.shape(train_batch_X[0])[1])

In [ ]:
print(type(list(zero_pad.shape)))

In [ ]:
print(type(np.shape(train_batch_X[0])[0]))

In [ ]:
print(type(list(np.shape(train_batch_X[0]))[0]))

In [ ]:
zero_pad[0,:12,:300]

In [ ]:
np.asarray(train_batch_X[1]).shape

In [ ]:
len(train_x)


In [ ]:
all_temp[0][1]

In [ ]:
temp_embedding

In [ ]:
data_set['news']

In [ ]:

embedding.wv.vocab

In [ ]:
label_set

In [ ]:
#data_label = {1:'culture',2:'global',3:'politic',4:'society',5:'economy'}

In [ ]:
kkm = Kkma()
kkm.pos(data_set['news'])

In [ ]:
len(data_set)

In [ ]:
for i in range(len(data_set)):
    twitter.pos(data_set['news'][i])

In [ ]:
konl = Twitter()
test_string =[

'test 데이터를 얼마나 잘 분류하는지 확인합니다.',
'학습이 epoch 단위로 진행될 때 예측력이 얼마나 상승하는지 ',
'loss에 대한 그래프를 그려서 확인합니다.',
'밑에 코드들은 그래프를 이쁘게 그려주기 위한 코드 입니다.'
]
for row in test_string:
    r = konl.pos(row, norm=True, stem=True)
    print( '=' * 20)
    for txt, post in r:
        print( txt, post)
    print( '=' * 20)



In [ ]:
# encoding: utf-8
import csv
import logging

from konlpy import jvm
from konlpy.tag import Twitter

logger = logging.getLogger(__name__)


def main():
    """
        konlpy 사용시 주의 사항
        자바 설치 및 세팅 필요
        JAVA_HOME 세팅이 필요합니다.
        export JAVA_HOME=$(/usr/libexec/java_home)
    """
    konl = Twitter()
    test_string = [
        u'konlpy 사용시 주의 사항',
        u'자바 설치 및 세팅 필요',
        u'JAVA_HOME 세팅이 필요합니다.',
        u'export JAVA_HOME=$(/usr/libexec/java_home)',
    ]
    for row in test_string:
        r = konl.pos(row, norm=True, stem=True)
        print( '=' * 20)
        for txt, post in r:
            print( txt, post)
        print( '=' * 20)


if __name__ == '__main__':
    jvm.init_jvm()
    main()

In [ ]:


model = Word2Vec(data_set['news'], min_count=3, iter=20, size=300,sg=1,
                window = 5)
model.init_sims(replace=True)